# PET 학습

In [1]:
import sys

In [2]:
sys.path.append(r'C:/Users/user/Desktop/논문리뷰/Pattern-Exploiting-Training(PET)/code/testPET - 복사본/src')
sys.path.append(r'C:/Users/user/Desktop/논문리뷰/Pattern-Exploiting-Training(PET)/code/testPET - 복사본/trained_pet모델')

In [3]:
import helper_functions
import pet.modeling
import pet.wrapper

In [4]:
import pandas as pd
import numpy as np

데이터 불러오기

In [5]:
df_train = pd.read_csv("C:/Users/user/Desktop/논문리뷰/Pattern-Exploiting-Training(PET)/code/testPET - 복사본/data3/df_train.csv")
df_test = pd.read_csv("C:/Users/user/Desktop/논문리뷰/Pattern-Exploiting-Training(PET)/code/testPET - 복사본/data3/df_test.csv")
df_unlabeled = pd.read_csv("C:/Users/user/Desktop/논문리뷰/Pattern-Exploiting-Training(PET)/code/testPET - 복사본/data3/df_unlabeled.csv")
df_eval = pd.read_csv("C:/Users/user/Desktop/논문리뷰/Pattern-Exploiting-Training(PET)/code/testPET - 복사본/data3/df_eval.csv")

In [6]:
df_train = df_train.astype(str)
df_test = df_test.astype(str)
df_unlabeled = df_unlabeled.astype(str)
df_eval = df_eval.astype(str)

In [7]:
label_list = np.array(['1', '2'])

개별 학습

In [8]:
pvp_numbers = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [23]:
# model_paths = []
# for i in pvp_numbers:
#     pvp_numbers = [i]
#     output_dir=f'C:/Users/user/Desktop/PET_trained_model/PVP{i}/'
#     model_path = helper_functions.train_models(pvp_numbers, df_train['text'], df_train['label'], df_eval['text'], df_eval['label'], 'yelp-polarity', label_list, output_dir=output_dir)
#     model_paths.append(model_path)

Training patterns: 100%|██████████| 1/1 [04:19<00:00, 259.47s/it]


In [8]:
model_paths = ['C:/Users/user/Desktop/PET_trained_model/PVP0/p0-i0',
                'C:/Users/user/Desktop/PET_trained_model/PVP1/p1-i0',
                'C:/Users/user/Desktop/PET_trained_model/PVP2/p2-i8',
                'C:/Users/user/Desktop/PET_trained_model/PVP3/p3-i1',
                'C:/Users/user/Desktop/PET_trained_model/PVP4/p4-i1',
                'C:/Users/user/Desktop/PET_trained_model/PVP5/p5-i9',
                'C:/Users/user/Desktop/PET_trained_model/PVP6/p6-i0',
                'C:/Users/user/Desktop/PET_trained_model/PVP7/p7-i4',
                'C:/Users/user/Desktop/PET_trained_model/PVP8/p8-i0',
                'C:/Users/user/Desktop/PET_trained_model/PVP9/p9-i0']

각 모델 별 점수 내기(scores 내는 데이터 1:test data, 2:unlabel data)

In [10]:
scores_test = helper_functions.score_models(
    model_paths, df_test['text'], df_test['label'], label_list
)

In [11]:
scores_unlabel = helper_functions.score_models(
    model_paths, df_unlabeled['text'], df_unlabeled['label'], label_list
)

In [9]:
scores_test = [0.8834, 0.9292, 0.9406, 0.9636, 0.9424, 0.9627, 0.9223, 0.7962, 0.5476, 0.8564]
scores_unlabel = [0.8842, 0.9341, 0.9403, 0.9637, 0.9463, 0.9639, 0.9273, 0.8056, 0.5426, 0.8554]

In [10]:
print('scores_test :', scores_test)
print('scores_unlabel :', scores_unlabel)

scores_test : [0.8834, 0.9292, 0.9406, 0.9636, 0.9424, 0.9627, 0.9223, 0.7962, 0.5476, 0.8564]
scores_unlabel : [0.8842, 0.9341, 0.9403, 0.9637, 0.9463, 0.9639, 0.9273, 0.8056, 0.5426, 0.8554]


추론 및 라벨링(개별)
score는 아무거나 해도 상관없음. 어차피 모델 1개라 앙상블 안함

In [11]:
labeling_accuracy_PVP = []

for i in range(0,10):
    model_path = [model_paths[i]]
    score_test = [scores_test[i]]
    
    unlabeled_probabilities = helper_functions.ensemble_predict(model_path, score_test, df_unlabeled['text'])
    acc = helper_functions.score_predictions(df_unlabeled['label'], unlabeled_probabilities, label_list)
    labeling_accuracy_PVP.append(acc)
    np.save(f'C:/Users/user/Desktop/논문리뷰/Pattern-Exploiting-Training(PET)/code/testPET - 복사본/data3/labeling/unlabeled_probabilities_PVP{i}.npy', unlabeled_probabilities)

In [12]:
labeling_accuracy_PVP

[0.8842,
 0.9341,
 0.9403,
 0.9637,
 0.9463,
 0.9639,
 0.9273,
 0.8056,
 0.5426,
 0.8554]

추론 및 라벨링(모델별)

unlabeled_probabilities_1 : scores_test로 가중치 설정하여 앙상블

unlabeled_probabilities_2 : scores_unlabel로 가중치 설정하여 앙상블

random1 = [0, 2, 4]

random2 = [1, 3, 6]

random3 = [5, 8, 9]

cosine = [3, 4, 5]

In [13]:
model_nums = [[0, 2, 4], [1, 3, 6], [5, 8, 9], [3, 4, 5]]

In [14]:
labeling_accuracy_test_model = []
labeling_accuracy_unlabel_model = []

In [15]:
for i in model_nums:
    model_num = i
    models_path = []
    models_score_test = []
    models_score_unlabel = []
    
    for j in model_num:
        model_path = model_paths[j]
        model_score_test = scores_test[j]
        model_score_unlabel = scores_unlabel[j]
        
        models_path.append(model_path)
        models_score_test.append(model_score_test)
        models_score_unlabel.append(model_score_unlabel)
        
    unlabeled_probabilities_1 = helper_functions.ensemble_predict(models_path, models_score_test, df_unlabeled['text'])
    acc_1 = helper_functions.score_predictions(df_unlabeled['label'], unlabeled_probabilities_1, label_list)
    labeling_accuracy_test_model.append(acc_1)
    np.save(f'C:/Users/user/Desktop/논문리뷰/Pattern-Exploiting-Training(PET)/code/testPET - 복사본/data3/labeling/unlabeled_probabilities_model{i}_test.npy', unlabeled_probabilities_1)
    
    unlabeled_probabilities_2 = helper_functions.ensemble_predict(models_path, models_score_unlabel, df_unlabeled['text'])
    acc_2 = helper_functions.score_predictions(df_unlabeled['label'], unlabeled_probabilities_2, label_list)
    labeling_accuracy_unlabel_model.append(acc_2)
    np.save(f'C:/Users/user/Desktop/논문리뷰/Pattern-Exploiting-Training(PET)/code/testPET - 복사본/data3/labeling/unlabeled_probabilities_model{i}_unlabel.npy', unlabeled_probabilities_2)    


In [16]:
labeling_accuracy_test_model

[0.9595, 0.9568, 0.9633, 0.9686]

In [17]:
labeling_accuracy_unlabel_model

[0.9595, 0.9567, 0.9632, 0.9686]